<a href="https://colab.research.google.com/github/21400126/Ionic/blob/master/1_NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nn.Module : contains layers

forward(input) : returns the output

Training procedure
Define the neural network that has some learnable parameters (or weights) -> Iterate over a dataset of inputs -> Process input through the network
-> Compute the loss (how far is the output from being correct) -> Propagate gradients back into the network’s parameters 
->Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    # nn.Conv2d will take in a 4D tensor of "nSamples * nChannels * height * width"
    self.conv1 = nn.Conv2d(1,6,3)       # 1input image channel, 6 output channels, 3x3 square convolution  
    self.conv2 = nn.Conv2d(6,16,3)
    
    self.fc1 = nn.Linear(16*6*6,120)       #6*6 from image dimension
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self,x):
    # Max pooling over a (2,2) window
    x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))       #???
    x = F.max_pool2d(F.relu|(self.conv2(x)),2)        # size가 squeare라면 single number 사용 가능
    
    x = x.view(-1,self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
  def num_flat_features(self, x):
    size = x.size()[1:]   
    num_features = 1
    for s in size :
      num_features *= s                     #?
    return num_features
    
net = Net()
print(net)

params = list(net.parameters())            #?
print(len(params))
print(params[0].size())                     #?
'''
input = torch.randn(1,1,32,32)              
out = net(input)                               # 왜?
print(out)
왜 안 되는지 몰겠음
'''

net.zero_grad()
out.backward(torch.randn(1,10))

Net(
  (conv1): Conv2d(1, 6, kernel_size=(32, 32), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(32, 32), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 32, 32])


RuntimeError: ignored